In [ ]:
import pandas as pd
import spacy

df_train= pd.read_csv("/Users/juan/Desktop/TFM/politicES_phase_2_train_public.csv", on_bad_lines='skip')
df_test= pd.read_csv("/Users/juan/Desktop/TFM/politicES_phase_2_test_codalab.csv", on_bad_lines='skip')
df_train.drop('label', axis=1, inplace=True)  
df_test.drop('label', axis=1, inplace=True)

In [ ]:
# Agrupar los tweets por 'label' y 'ideology_multiclass' y luego concatenar los tweets
df_train= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()
df_test= df_train.groupby(['label', 'ideology_multiclass'])['tweet'].apply(' '.join).reset_index()

In [2]:
nlp = spacy.load('es_core_news_sm')

def preprocess_text(tweet):
    # Procesa el tweet usando el modelo de spaCy
    doc = nlp(tweet)
    # Lematiza y elimina stopwords
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return ' '.join(tokens)

In [3]:
df_train['clean_tweet'] = df_train['tweet'].apply(preprocess_text)
df_test['clean_tweet'] = df_test['tweet'].apply(preprocess_text)

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score

# Suponiendo que df_train y df_test son tus DataFrames

# Inicializar el vectorizador TF-IDF
vectorizador = TfidfVectorizer()

# Ajustar el vectorizador al conjunto de entrenamiento y transformarlo
X_train_tfidf = vectorizador.fit_transform(df_train['clean_tweet'])
# Transformar el conjunto de prueba usando el mismo vectorizador
X_test_tfidf = vectorizador.transform(df_test['clean_tweet'])

# Definir las etiquetas de entrenamiento y prueba
y_train = df_train['ideology_multiclass']
y_test = df_test['ideology_multiclass']

# Crear e instanciar el modelo de regresión logística
logistic_model = LogisticRegression(solver='liblinear')  # Usando liblinear que es bueno para conjuntos de datos pequeños y medianos
logistic_model.fit(X_train_tfidf, y_train)

# Realizar la predicción
y_pred = logistic_model.predict(X_test_tfidf)

# Calcular y mostrar el F1 score y el informe de clasificación
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(macro_f1)
print(classification_report(y_test, y_pred))
